## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()

# Display the unique values for each column
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Display the value counts
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500

# Create a list of application types to replace
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display the classification counts
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_filtered = classification_counts[classification_counts > 1]

# Display the filtered classification counts
print(classification_counts_filtered)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 100

# Create a list of classifications to replace
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`

# Check data types and number of unique values
categorical_candidates = application_df.select_dtypes(include=['object']).columns.tolist()

# Add columns with few unique values (e.g., less than 10 unique values) to the list
additional_categorical = application_df.nunique()[application_df.nunique() < 10].index.tolist()

# Combine the two lists
categorical_columns = list(set(categorical_candidates + additional_categorical))

# Display the categorical columns
print("Potential categorical variables:", categorical_columns)

application_df = pd.get_dummies(application_df, columns=[
    'APPLICATION_TYPE', 
    'INCOME_AMT', 
    'AFFILIATION', 
    'USE_CASE', 
    'CLASSIFICATION', 
    'IS_SUCCESSFUL', 
    'STATUS', 
    'ORGANIZATION', 
    'SPECIAL_CONSIDERATIONS'
], drop_first=True)

# Display the first few rows of the updated DataFrame
application_df.head()

Potential categorical variables: ['CLASSIFICATION', 'INCOME_AMT', 'APPLICATION_TYPE', 'ORGANIZATION', 'USE_CASE', 'STATUS', 'IS_SUCCESSFUL', 'SPECIAL_CONSIDERATIONS', 'AFFILIATION']


,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,INCOME_AMT_1-9999,...,CLASSIFICATION_C4000,CLASSIFICATION_C5000,CLASSIFICATION_C7000,CLASSIFICATION_Other,IS_SUCCESSFUL_1,STATUS_1,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_Y
0,5000,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,False
1,108590,False,False,True,False,False,False,False,False,True,...,False,False,False,False,True,True,True,False,False,False
2,5000,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,6692,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,True,False
4,142590,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,True,False


In [10]:
# Split our preprocessed data into our features and target arrays

# Define the target variable (y)
y = application_df['IS_SUCCESSFUL_1']

# Define the features (X) by dropping the target column
X = application_df.drop(columns=['IS_SUCCESSFUL_1'])

# Display the shapes of X and y to confirm the split
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the training and testing datasets
print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)

Features shape: (34299, 42)
Target shape: (34299,)
Training features shape: (27439, 42)
Testing features shape: (6860, 42)
Training target shape: (27439,)
Testing target shape: (6860,)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print("Scaled training features shape:", X_train_scaled.shape)
print("Scaled testing features shape:", X_test_scaled.shape)

Scaled training features shape: (27439, 42)
Scaled testing features shape: (6860, 42)


## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,865 (19.00 KB)

 Trainable params: 4,865 (19.00 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the model
nn.compile(optimizer='adam', 
           loss='binary_crossentropy', 
           metrics=['accuracy'])

In [14]:
# Train the model
history = nn.fit(X_train_scaled, y_train, 
                 epochs=100, 
                 batch_size=32, 
                 validation_split=0.2, 
                 verbose=1)

Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 541us/step - accuracy: 0.6777 - loss: 0.6155 - val_accuracy: 0.7343 - val_loss: 0.5527
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step - accuracy: 0.7270 - loss: 0.5585 - val_accuracy: 0.7396 - val_loss: 0.5549
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - accuracy: 0.7278 - loss: 0.5542 - val_accuracy: 0.7363 - val_loss: 0.5443
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step - accuracy: 0.7318 - loss: 0.5459 - val_accuracy: 0.7362 - val_loss: 0.5457
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step - accuracy: 0.7304 - loss: 0.5487 - val_accuracy: 0.7385 - val_loss: 0.5437
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step - accuracy: 0.7343 - loss: 0.5438 - val_accuracy: 0.7392 - val_loss: 0.5438
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.7274 - loss: 0.5500 - val_accuracy: 0.7380 - val_loss: 0.5425
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 478us/step - accuracy: 0.7279 - loss: 0

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 316us/step - accuracy: 0.7249 - loss: 0.5620
Loss: 0.5619959831237793, Accuracy: 0.7249271273612976


In [16]:
# Export our model to HDF5 file
nn.save('trained_model.h5')